# Assignment 1 

Database: (50_Startups)

You are being hired as a Data Scientist client wants
to create a model that can predict the profit of
the company based on company's location and Rd expense

Client expects the minimum accuracy of 0.92 of the deployed model

Your job is to check the feasibility of the project and create best training 
and testing splits


In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('50_Startups.csv')

In [4]:
data.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


# Data Preprocessing

In [7]:
data.isna().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [9]:
data['R&D Spend'].fillna(round(data['R&D Spend'].mean()), inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [11]:
data['Marketing Spend'].fillna(round(data['Marketing Spend'].mean()), inplace=True)

In [12]:
data['State'].fillna(data['State'].mode()[0], inplace=True)

In [19]:
#features = data.iloc[:,:-1].values
#label = data.iloc[:,-1].values

In [35]:
# Categorical Data Handling
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)
fState = ohe.fit_transform(np.array(data['State']).reshape(-1,1))
features = np.concatenate([fState, np.array(data.iloc[:, [0]])], axis=1)

In [36]:
label = data.iloc[:, -1].values
label

array([192261.83, 191792.06, 191050.39, 182901.99, 166187.94, 156991.12,
       156122.51, 155752.6 , 152211.77, 149759.96, 146121.95, 144259.4 ,
       141585.52, 134307.35, 132602.65, 129917.04, 126992.93, 125370.37,
       124266.9 , 122776.86, 118474.03, 111313.02, 110352.25, 108733.99,
       108552.04, 107404.34, 105733.54, 105008.31, 103282.38, 101004.64,
        99937.59,  97483.56,  97427.84,  96778.92,  96712.8 ,  96479.51,
        90708.19,  89949.14,  81229.06,  81005.76,  78239.91,  77798.83,
        71498.49,  69758.98,  65200.33,  64926.08,  49490.75,  42559.73,
        35673.41,  14681.4 ])

In [37]:
from sklearn.neighbors import KNeighborsRegressor
modelTest = KNeighborsRegressor()

In [38]:
from sklearn.model_selection import cross_val_score

# Supress warnings
import warnings
warnings.filterwarnings('ignore')

scores = cross_val_score(modelTest,
                        features,
                        label,
                        cv = 10)

scores

array([ -9.52847973,  -2.42061691,  -7.28965118, -22.85356329,
        -0.37428142,  -8.95243978, -33.5713245 ,  -1.3902268 ,
        -1.76905992,  -3.65968395])

# Method 1 --- Using GridSearchCV

In [30]:
'''

KNeighborsRegressor(               
n_neighbors=5,	               Positive Integer
*,
weights='uniform',             ‘uniform’, 'distance'
algorithm='auto',              ‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’
leaf_size=30,                  Positive Integer
p=2,                           1 (Manhattan distance),   2(Elucidean distance)
metric='minkowski',            "“euclidean”, “manhattan”, “chebyshev”, “minkowski”
                               “wminkowski”, “seuclidean”, “mahalanobis”"
metric_params=None,            Dictionary
n_jobs=None                    -1, 1 or -2
)

'''

'\n\nKNeighborsRegressor(               \nn_neighbors=5,\t               Positive Integer\n*,\nweights=\'uniform\',             ‘uniform’, \'distance\'\nalgorithm=\'auto\',              ‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’\nleaf_size=30,                  Positive Integer\np=2,                           1 (Manhattan distance),   2(Elucidean distance)\nmetric=\'minkowski\',            "“euclidean”, “manhattan”, “chebyshev”, “minkowski”\n                               “wminkowski”, “seuclidean”, “mahalanobis”"\nmetric_params=None,            Dictionary\nn_jobs=None                    -1, 1 or -2\n)\n\n'

In [39]:
#Step1: Design the parameter grid. In Python, you can represent parameter grid in the form of Dictionary

weightsParameter = ['uniform','distance']
n_neighborsParameter = np.arange(1,31)
algorithmParameter = ['auto', 'ball_tree', 'kd_tree', 'brute']
metricParameter = ['euclidean', 'manhattan', 'chebyshev' , 'minkowski', 'seuclidean', 'mahalanobis']


paramGrid = dict(n_neighbors = n_neighborsParameter,
                weights= weightsParameter,
                algorithm = algorithmParameter,
                metric=metricParameter)

# Step2: Initialize the algo

from sklearn.neighbors import KNeighborsRegressor
modelGridSearch = KNeighborsRegressor()

#Step3: Search the best parameter for your data

#GridSearchCV


from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(modelGridSearch,
                   param_grid=paramGrid,
                   cv = 10) #Same as cross_val_score

In [40]:
# Step4: Extract Results

grid.fit(features,label)

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'metric': ['euclidean', 'manhattan', 'chebyshev',
                                    'minkowski', 'seuclidean', 'mahalanobis'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
                         'weights': ['uniform', 'distance']})

In [41]:
grid.best_score_

-9.180932748418233

In [45]:
grid.best_estimator_

KNeighborsRegressor(metric='euclidean')

In [44]:
grid.best_params_

{'algorithm': 'auto',
 'metric': 'euclidean',
 'n_neighbors': 5,
 'weights': 'uniform'}

In [46]:
finalModel = KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')